In [1]:
# Option 1
# %run ./Spec_UI.py

from Spec_UI import SpecificationSetupUI
ui = SpecificationSetupUI()


In [2]:
# ui.save_to_json()  # Manual save
# ui.reset_all()     # Reset everything (pass None as required argument)
config = ui.current_config()  # Get current state

In [3]:
print(ui.current_config())

{'Metadata': {'num_processors': 4, 'debug_output_frequency': 10}, 'PottsCore': {'dim_x': 100, 'dim_y': 100, 'dim_z': 1, 'steps': 100000, 'anneal': 0, 'fluctuation_amplitude': 10.0, 'fluctuation_amplitude_function': 'Min', 'boundary_x': 'Periodic', 'boundary_y': 'Periodic', 'boundary_z': 'NoFlux', 'neighbor_order': 2, 'debug_output_frequency': 0, 'random_seed': None, 'lattice_type': 'Cartesian', 'offset': 0, 'energy_function_calculator': None}, 'CellType': [{'Cell type': 'T1', 'freeze': False}, {'Cell type': 'T2', 'freeze': False}, {'Cell type': 'T3', 'freeze': False}], 'Constraints': {'Volume': [{'CellType': 'Medium', 'enabled': False, 'target_volume': 25.0, 'lambda_volume': 2.0}], 'Surface': []}}


## Adhesion Demo

In [4]:
from cc3d.core.PyCoreSpecs import Metadata, PottsCore

# dim_x = dim_y = 100
specs = [
    # Metadata(num_processors=4),
    Metadata(num_processors=config['Metadata']['num_processors']),             
    PottsCore(dim_x=config['PottsCore']['dim_x'],
              dim_y=config['PottsCore']['dim_y'],
              neighbor_order=config['PottsCore']['neighbor_order'],
              boundary_x=config['PottsCore']['boundary_x'],
              boundary_y=config['PottsCore']['boundary_y'])
            #   neighbor_order=2,
            #   boundary_x="Periodic",
            #   boundary_y="Periodic")
]

In [5]:
from cc3d.core.PyCoreSpecs import CellTypePlugin

cell_types = [cell['Cell type'] for cell in config['CellType']]
# cell_types = ["T1", "T2", "T3"]
print(cell_types)
# specs.append(CellTypePlugin(*cell_types))

['T1', 'T2', 'T3']


In [ ]:
from cc3d.core.PyCoreSpecs import VolumePlugin

volume_specs = VolumePlugin()
# for ct in cell_types:
    # volume_specs.param_new(ct, target_volume=25, lambda_volume=2)
for cell in config['Constraints']['Volume']]:
    volume_specs.param_new(cell['CellType'], target_volume=cell['target_volume'], lambda_volume=cell['lambda_volume'])
specs.append(volume_specs)

In [ ]:
from cc3d.core.PyCoreSpecs import ContactPlugin, AdhesionFlexPlugin

# contact_specs = ContactPlugin(neighbor_order=2)
contact_specs = ContactPlugin(neighbor_order=config['PottsCore']['neighbor_order'])
for idx1 in range(len(cell_types)):
    contact_specs.param_new(type_1="Medium", type_2=cell_types[idx1], energy=16)
    for idx2 in range(idx1, len(cell_types)):
        contact_specs.param_new(type_1=cell_types[idx1], type_2=cell_types[idx2], energy=16)
specs.append(contact_specs)

adhesion_specs = AdhesionFlexPlugin(neighbor_order=2)
adhesion_specs.density_new(molecule="M1", cell_type="T1", density=1.0)
adhesion_specs.density_new(molecule="M2", cell_type="T2", density=1.0)
formula = adhesion_specs.formula_new(formula_name='Binary')
formula.param_set("M1", "M1", -10.0)
formula.param_set("M1", "M2", 0.0)
formula.param_set("M2", "M2", 10.0)
specs.append(adhesion_specs)

In [10]:
from cc3d.core.PyCoreSpecs import UniformInitializer

unif_init_specs = UniformInitializer()
unif_init_specs.region_new(width=5, pt_min=(0, 0, 0), pt_max=(dim_x, dim_y, 1),
                           cell_types=["T1", "T1", "T2", "T2", "T3"])
specs.append(unif_init_specs)

In [11]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService

cc3d_sim = CC3DSimService()
cc3d_sim.register_specs(specs)
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()

WILL RUN SIMULATION FROM BEGINNING


/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: PlayerSizes
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: PlayerSizesFloating
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: RecentSimulations
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
Random number generator: MersenneTwister


True

In [12]:
import ipywidgets

def _cb_m1_m1(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M1']['M1'] = change.new
        adhesion_specs.steer()

def _cb_m1_m2(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M1']['M1'] = change.new
        adhesion_specs.steer()

def _cb_m2_m2(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M2']['M2'] = change.new
        adhesion_specs.steer()

slider_m1_m1 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M1']['M1'], 
    min=-10, 
    max=10, 
    step=0.1, 
    continuous_update=False, 
    description='M1-M1'
)
slider_m1_m2 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M1']['M2'], 
    min=-10, 
    max=10, 
    step=0.1, 
    continuous_update=False, 
    description='M1-M2'
)
slider_m2_m2 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M2']['M2'], 
    min=-10, 
    max=10, 
    step=0.1, 
    continuous_update=False, 
    description='M2-M2'
)

slider_m1_m1.observe(_cb_m1_m1, names='value')
slider_m1_m2.observe(_cb_m1_m2, names='value')
slider_m2_m2.observe(_cb_m2_m2, names='value')

In [13]:
from IPython.display import display

cc3d_sim.visualize(plot_freq=10).show()
display(slider_m1_m1)
display(slider_m1_m2)
display(slider_m2_m2)
display(cc3d_sim.jupyter_run_button())

CC3DViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

FloatSlider(value=-10.0, continuous_update=False, description='M1-M1', max=10.0, min=-10.0)

FloatSlider(value=0.0, continuous_update=False, description='M1-M2', max=10.0, min=-10.0)

FloatSlider(value=10.0, continuous_update=False, description='M2-M2', max=10.0, min=-10.0)

ToggleButton(value=False, description='Run Simulation')